# CitiBike Destination Predictor

First we import 12 months of data from 2018. 


In [2]:
import pandas as pd

In [220]:
for i in range(1,13):
    if i == 1:
        big = pd.read_csv('citibike_data/2018'+ str(i).zfill(2)+'-citibike-tripdata.csv')
        big = big.sample(frac=.1)
    else:
        temp = pd.read_csv('citibike_data/2018'+ str(i).zfill(2)+'-citibike-tripdata.csv')
        big = pd.concat([big, temp.sample(frac=.1)])

### NAN values

There are just 260 NaN values missing the station start and end points in the file of 1.8million. This information is crucial to the prediction and thus can not be substituted. We have decidied to simply remove these rows. 

In [221]:
big = big.dropna()

In [222]:
big.shape

(1754542, 15)

In [223]:
big.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
395157,556,2018-01-02 07:50:24.9430,2018-01-02 07:59:41.4310,518.0,E 39 St & 2 Ave,40.747804,-73.973442,524.0,W 43 St & 6 Ave,40.755273,-73.983169,21208,Subscriber,1965,1
438792,689,2018-01-22 11:47:21.0270,2018-01-22 11:58:50.1400,539.0,Metropolitan Ave & Bedford Ave,40.715348,-73.960241,3077.0,Stagg St & Union Ave,40.708771,-73.950953,20996,Subscriber,1988,1
172138,1649,2018-01-21 11:36:39.5720,2018-01-21 12:04:08.8680,366.0,Clinton Ave & Myrtle Ave,40.693261,-73.968896,3373.0,3 St & 3 Ave,40.675071,-73.987752,16871,Subscriber,1982,2
195961,467,2018-01-27 22:35:54.0940,2018-01-27 22:43:41.2360,387.0,Centre St & Chambers St,40.712733,-74.004607,3467.0,W Broadway & Spring Street,40.724947,-74.001659,21089,Customer,1969,0
440661,665,2018-01-22 13:50:51.7820,2018-01-22 14:01:57.7380,540.0,Lexington Ave & E 29 St,40.743116,-73.982154,161.0,LaGuardia Pl & W 3 St,40.729170,-73.998102,17034,Subscriber,1982,1


### convert to DateTime

In [224]:
big['starttime'] = pd.to_datetime(big['starttime'])
big['start_month'] = big.starttime.dt.month
big['start_day_of_week'] = big.starttime.dt.dayofweek
big['start_hour'] = big.starttime.dt.hour
big['start_date'] = big.starttime.dt.day

big['starttime'] = big.starttime.dt.date
big['starttime'] = pd.to_datetime(big['starttime'])


### Insert Weather Data Here: 

https://www.weather.gov/okx/centralparkhistorical

In [225]:
weather_df = pd.read_csv('citibike_data/2018_weather.csv')
weather_df['starttime'] = pd.to_datetime(weather_df['date'])
weather_df = weather_df.drop(['max_temp', 'min_temp','departure', 'Hdd', 'cdd', 'new_snow', 'date' ], axis=1)                                               

In [226]:
big = pd.merge(big, weather_df, on='starttime', how='left')

### Insert Neighborhood Name

Connect Neighborhood name data to our 'End Station Latitude/Longitude' to use as labels for predictions. 

In [227]:
neighborhoods = pd.read_csv('citibike_data/citibike_neighborhoods.csv')

In [228]:
neighborhoods['end station name'] = neighborhoods['stops']
neighborhoods = neighborhoods.drop(['Unnamed: 0', 'stops', 'end station latitude', 'end station longitude'], axis=1)

In [229]:
big = pd.merge(big, neighborhoods, on='end station name', how='left')

In [230]:
big.columns

Index(['tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender', 'start_month', 'start_day_of_week',
       'start_hour', 'start_date', 'ave_temp', 'precip', 'snow_depth',
       'stop_nhbr'],
      dtype='object')

### convert usertype to categories

In [231]:
big.usertype = pd.Categorical(big.usertype)
big.usertype = big.usertype.cat.codes

### Remove redundant rows.

In [232]:
big = big.drop(['starttime', 'stoptime','start station name',
                'end station id', 'bikeid', 'start station latitude', 
                'start station longitude', 'end station latitude', 'end station longitude','end station name', 'start_date'], axis=1)


Questions: 

How do we handle start time? 

Do we give a category label to Start Station? 



In [238]:
big.to_csv('citibike_2018.csv', index=False)

In [237]:
big.sample(100)

,tripduration,start station id,usertype,birth year,gender,start_month,start_day_of_week,start_hour,ave_temp,precip,snow_depth,stop_nhbr
567622,1728,327.0,0,1994,2,6,6,12,58.5,0.25,0,Chelsea
194551,680,396.0,1,1967,1,3,0,8,37.5,0.08,0,Columbus Circle
1218724,850,3495.0,1,1990,1,9,5,14,68.0,0.00,0,Clinton
1284159,114,2002.0,0,1969,0,9,5,10,66.0,0.02,0,East Village
549562,64,504.0,1,1965,1,5,6,17,61.0,0.00,0,Park Slope
1231261,790,3235.0,1,1988,1,9,5,13,68.0,0.00,0,Williamsburg
191782,433,435.0,1,1998,2,3,4,8,53.0,0.03,0,Tribeca
1181033,361,3686.0,1,1966,1,9,4,17,69.5,0.00,0,Upper East Side
1259665,614,461.0,1,1968,1,9,3,14,68.5,0.00,0,Clinton
354986,219,3414.0,1,1991,1,4,5,14,43.0,0.00,0,Park Slope
